In [16]:
import sys

sys.path.insert(1, "/home/lucas/harmonizacaoAutomatica/src/")

In [1]:
import os
import glob
import csv
import numpy as np
from keras.models import load_model
from src.system.load_data import carrega_arquivo
from src.analysis.performance_measures import (
    calc_accuracy,
    calc_log_loss,
)

main = "src/system/results/lstm_1671757520/"
caminho = main + "modelos/*.h5"

_, _, _, _, X_test, Y_test = carrega_arquivo()
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

arquivos_h5 = glob.glob(caminho)
arquivos_h5[0:10]

['src/system/results/lstm_1671757520/modelos/28.h5',
 'src/system/results/lstm_1671757520/modelos/22.h5',
 'src/system/results/lstm_1671757520/modelos/9.h5',
 'src/system/results/lstm_1671757520/modelos/29.h5',
 'src/system/results/lstm_1671757520/modelos/12.h5',
 'src/system/results/lstm_1671757520/modelos/8.h5',
 'src/system/results/lstm_1671757520/modelos/2.h5',
 'src/system/results/lstm_1671757520/modelos/11.h5',
 'src/system/results/lstm_1671757520/modelos/17.h5',
 'src/system/results/lstm_1671757520/modelos/23.h5']

In [3]:
for rodada, arquivo in enumerate(arquivos_h5):
    if os.path.isfile(main + "output/predicao_por_fold.npy"):
        predicao_por_fold = np.load(main + "output/predicao_por_fold.npy")
        real_por_fold = np.load(main + "output/real_por_fold.npy")
    else:
        predicao_por_fold = []
        real_por_fold = []


    modelo = load_model(arquivo)
    predicao = modelo.predict(X_test)

    loss = calc_log_loss(Y_test, np.nan_to_num(predicao))
    acc = calc_accuracy(Y_test, np.nan_to_num(predicao))

    log = open(main + "log_resultados.csv", "a")
    log.write(f"\n{rodada+1},{loss},{acc}")
    log.close()

    if len(predicao_por_fold) == 0:
        predicao_por_fold.append(np.array(predicao))
        real_por_fold.append(np.array(Y_test))
    else:
        predicao_por_fold = np.append(predicao_por_fold, [np.array(predicao)], axis=0)
        real_por_fold = np.append(real_por_fold, [np.array(Y_test)], axis=0)

    np.save(main + "output/predicao_por_fold.npy", predicao_por_fold)
    np.save(main + "output/real_por_fold.npy", real_por_fold)

114/114 [==============================] - 0s 1ms/step
